In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
salary  = pd.read_csv('../final/salary.csv')
salary.columns

# officer date - start date at which officer became officer according to this dataset - ignore for now
# salary per year per position (median + bars)
# salary per year per seniority (median of binning)